In [24]:
import pandas as pd
from urllib.parse import quote
import requests
import json

response_data = []
title_year_set = set()

api_key = 'b5546873'
api_url = 'http://www.omdbapi.com/?t={}&y={}&apikey={}'

budget_data = pd.read_csv('top_grossing_movies.csv')
titles_and_years = budget_data[['highest_grossing_year', 'Title']]

for index, row in titles_and_years.iterrows():
    title_year_set.add((row['Title'], row['highest_grossing_year']))

    
title_year_set = sorted(title_year_set)
counter = 0
for title, year in title_year_set:
    if counter < 4500:
        counter += 1
        continue
    if counter == 5000:
        break
    
    response_dict = dict()
    uri_title = quote(title)
    url = api_url.format(uri_title, year, api_key)
    response = requests.get(url)
    try:
        response_dict = json.loads(response.text)
    except:
        print('failed to parse for movie ', title)
        continue
    
    if response_dict['Response'] == 'False':
        prev_year = year - 1
        url = api_url.format(uri_title, prev_year, api_key)
        response = requests.get(url)
        try:
            response_dict = json.loads(response.text)
        except:
            print('failed to parse for movie ', title)
            continue
        response_dict = json.loads(response.text)
    
    response_dict['Title'] = title
    response_data.append(response_dict)
    counter += 1
    
response_data

[{'Title': 'War for the Planet of the Apes',
  'Year': '2017',
  'Rated': 'PG-13',
  'Released': '14 Jul 2017',
  'Runtime': '140 min',
  'Genre': 'Action, Adventure, Drama, Sci-Fi, Thriller',
  'Director': 'Matt Reeves',
  'Writer': 'Mark Bomback, Matt Reeves, Rick Jaffa (based on characters created by), Amanda Silver (based on characters created by)',
  'Actors': 'Andy Serkis, Woody Harrelson, Steve Zahn, Karin Konoval',
  'Plot': 'After the apes suffer unimaginable losses, Caesar wrestles with his darker instincts and begins his own mythic quest to avenge his kind.',
  'Language': 'English, American Sign Language',
  'Country': 'USA, Canada, New Zealand',
  'Awards': 'Nominated for 1 Oscar. Another 22 wins & 46 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNDNmYTQzMDEtMmY0MS00OTNjLTk4MjItMDZhMzkzOGI3MzA0XkEyXkFqcGdeQXVyNjk5NDA3OTk@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '

In [8]:
imdb_df = pd.DataFrame(response_data)
imdb_df.to_csv('imdb_budget.csv', mode='a', header=False)

In [7]:
import pandas as pd
from urllib.parse import quote
import requests
import json

response_data = []
title_year_set = set()

api_key = '6d4eee8d'
api_url = 'http://www.omdbapi.com/?t={}&y={}&apikey={}'

failed_df = pd.read_csv('failed_downloads.csv')
failed_df

titles_and_years = failed_df[['title', 'year']]
for index, row in titles_and_years.iterrows():
    title_year_set.add((row['title'], row['year']))

    
title_year_set = sorted(title_year_set)
counter = 0
for title, year in title_year_set:
    if counter < 0:
        counter += 1
        continue
    if counter == 500:
        break
    
    response_dict = dict()
    uri_title = quote(title)
    url = api_url.format(uri_title, year, api_key)
    response = requests.get(url)
    try:
        response_dict = json.loads(response.text)
    except:
        print('failed to parse for movie ', title)
        continue
    
    if response_dict['Response'] == 'False':
        prev_year = year - 1
        url = api_url.format(uri_title, prev_year, api_key)
        response = requests.get(url)
        try:
            response_dict = json.loads(response.text)
        except:
            print('failed to parse for movie ', title)
            continue
        response_dict = json.loads(response.text)
    
    response_dict['Title'] = title
    response_data.append(response_dict)
    counter += 1
    
response_data

[{'Title': '...And God Created Woman',
  'Year': '1956',
  'Rated': 'PG',
  'Released': '21 Oct 1957',
  'Runtime': '90 min',
  'Genre': 'Drama, Romance',
  'Director': 'Roger Vadim',
  'Writer': 'Roger Vadim (original scenario), Raoul Lévy (original scenario)',
  'Actors': 'Brigitte Bardot, Curd Jürgens, Jean-Louis Trintignant, Jane Marken',
  'Plot': 'In sunny St. Tropez, a young sexpot loves one brother but marries the other.',
  'Language': 'French, English',
  'Country': 'France, Italy',
  'Awards': '1 nomination.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTkyMDA4Mjc0Nl5BMl5BanBnXkFtZTgwMzgwODgxNDE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '75%'}],
  'Metascore': 'N/A',
  'imdbRating': '6.3',
  'imdbVotes': '6,777',
  'imdbID': 'tt0049189',
  'Type': 'movie',
  'DVD': '18 Jul 2000',
  'BoxOffice': 'N/A',
  'Production': 'Criterion Collection',
  'Website': 'N/A',
  'Response': 'Tr